In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import time
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, LSTM, BatchNormalization, CuDNNLSTM, CuDNNGRU, Activation, Flatten
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import SGD

In [3]:
from collections import deque

In [4]:
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression

In [5]:
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('Data/data.dat', sep=' ')

In [7]:
gvkey_dict = {}

loop_start = time.time()

for gvkey in df['gvkey'].unique()[:50]:
    stock_start = time.time()
    df_temp = df[df['gvkey'] == 1266]
    df_temp['year'] = df_temp['year'].map(lambda x: str(x))
    df_temp['month'] = df_temp['month'].map(lambda x: str(x))
    df_temp['-'] = '-'
    df_temp['timestemp'] = df_temp['year'] + df_temp['-'] + df_temp['month']
    df_temp['timestemp'] = pd.to_datetime(df_temp['timestemp'])
    df_temp = df_temp.set_index('timestemp')
    df_temp = df_temp.drop(['active', 'year', 'month', '-', 'date'], axis=1)
    df_temp['EBIT'] = df_temp['saleq_ttm'] - df_temp['cogsq_ttm'] - df_temp['xsgaq_ttm']
    df_temp['EBIT/EV'] = df_temp['EBIT'] / df_temp['entval']
    
    featureList = ['mom1m','mom3m','cogsq_ttm', 'xsgaq_ttm', 'oiadpq_ttm',
       'niq_ttm', 'cheq_mrq', 'rectq_mrq', 'invtq_mrq', 'acoq_mrq',
       'ppentq_mrq', 'aoq_mrq', 'dlcq_mrq', 'apq_mrq', 'txpq_mrq', 'lcoq_mrq',
       'ltq_mrq', 'seqq_mrq', 'atq_mrq', 'csho_1yr_avg', 'adjusted_price',
       'prccm', 'ajexm']

    df_temp.drop(['gvkey', 'gics-sector'], axis=1, inplace=True)
    df_temp = df_temp[~df_temp.isin([np.nan, np.inf, -np.inf]).any(1)]
    df_temp.dropna(inplace=True)
    
    periods = 12
    
    min_max_scaler = preprocessing.MinMaxScaler()
    df_temp = pd.DataFrame(min_max_scaler.fit_transform(df_temp), columns=df_temp.columns.values)

    df_temp['Y'] = df_temp.iloc[:, -1].shift(periods=-periods)
    df_temp.dropna(inplace=True)
    
    seq_df = []
    prev_months = deque(maxlen=12)

    for i in df_temp.values:
        prev_months.append(i)
        if len(prev_months) == 12:
            seq_df.append(np.array(prev_months))  

    seq_df = np.array(seq_df)
    
    Y = seq_df[:, -1, -1]
    X = seq_df[:, :, :-1]

    display(X.shape[1:])
    X_temp = X.reshape((X.shape[0], X.shape[1]*X.shape[2]))

    display(X_temp[0,[29,59,89]])
    
    X_train = X_temp[:int(len(X)*0.7)]
    X_test = X_temp[int(len(X)*0.7):]

    Y_train = Y[:int(len(X)*0.7)]
    Y_test = Y[int(len(X)*0.7):]
    
    display(X_train.shape)
    model = Sequential()
    model.add(Dense(1, input_shape=(360,)))
    model.add(Activation('linear'))

    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    model.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test))

    Y_pred = model.predict(X_test)
    
    gvkey_dict[gvkey] = Y_pred[:,0]
    stock_end = time.time()
    print("It has been {0} seconds since this iteration started".format(stock_end - stock_start))
    
loop_end = time.time()
print("It has been {0} seconds since the loop started".format(loop_end - loop_start))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

(12, 30)

array([0.00899663, 0.08226673, 0.08325124])

(287, 360)

Train on 287 samples, validate on 124 samples
Epoch 1/100
287/287 [==============================] - 0s 509us/step - loss: 0.0723 - mean_squared_error: 0.0723 - val_loss: 0.1257 - val_mean_squared_error: 0.1257
Epoch 2/100
287/287 [==============================] - 0s 37us/step - loss: 0.0460 - mean_squared_error: 0.0460 - val_loss: 0.1172 - val_mean_squared_error: 0.1172
Epoch 3/100
287/287 [==============================] - 0s 38us/step - loss: 0.0328 - mean_squared_error: 0.0328 - val_loss: 0.1252 - val_mean_squared_error: 0.1252
Epoch 4/100
287/287 [==============================] - 0s 33us/step - loss: 0.0290 - mean_squared_error: 0.0290 - val_loss: 0.1367 - val_mean_squared_error: 0.1367
Epoch 5/100
287/287 [==============================] - 0s 35us/step - loss: 0.0268 - mean_squared_error: 0.0268 - val_loss: 0.1347 - val_mean_squared_error: 0.1347
Epoch 6/100
287/287 [==============================] - ETA: 0s - loss: 0.0201 - mean_squared_error: 0.02 - 0s 29us/step - loss: 0.025

...

array([0.00899663, 0.08226673, 0.08325124])

(287, 360)

Train on 287 samples, validate on 124 samples
Epoch 1/100
287/287 [==============================] - 1s 4ms/step - loss: 0.0589 - mean_squared_error: 0.0589 - val_loss: 0.1590 - val_mean_squared_error: 0.1590
Epoch 2/100
287/287 [==============================] - 0s 51us/step - loss: 0.0333 - mean_squared_error: 0.0333 - val_loss: 0.0867 - val_mean_squared_error: 0.0867
Epoch 3/100
287/287 [==============================] - 0s 56us/step - loss: 0.0295 - mean_squared_error: 0.0295 - val_loss: 0.0901 - val_mean_squared_error: 0.0901
Epoch 4/100
287/287 [==============================] - 0s 62us/step - loss: 0.0256 - mean_squared_error: 0.0256 - val_loss: 0.0919 - val_mean_squared_error: 0.0919
Epoch 5/100
287/287 [==============================] - 0s 59us/step - loss: 0.0249 - mean_squared_error: 0.0249 - val_loss: 0.0919 - val_mean_squared_error: 0.0919
Epoch 6/100
287/287 [==============================] - 0s 61us/step - loss: 0.0241 - mean_squared_error: 0.0241 - val_loss: 0.0944 - va

287/287 [==============================] - 0s 64us/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.1437 - val_mean_squared_error: 0.1437
Epoch 51/100
287/287 [==============================] - 0s 55us/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.1192 - val_mean_squared_error: 0.1192
Epoch 52/100
287/287 [==============================] - 0s 61us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.1254 - val_mean_squared_error: 0.1254
Epoch 53/100
287/287 [==============================] - 0s 61us/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.1672 - val_mean_squared_error: 0.1672
Epoch 54/100
287/287 [==============================] - 0s 55us/step - loss: 0.0078 - mean_squared_error: 0.0078 - val_loss: 0.1541 - val_mean_squared_error: 0.1541
Epoch 55/100
287/287 [==============================] - 0s 59us/step - loss: 0.0075 - mean_squared_error: 0.0075 - val_loss: 0.1319 - val_mean_squared_error: 0.1319
Epoch 56/100
287/287 [=

Epoch 100/100
287/287 [==============================] - 0s 88us/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.2035 - val_mean_squared_error: 0.2035
It has been 4.878367900848389 seconds since this iteration started
It has been 145.48865294456482 seconds since the loop started


In [11]:
df_pred = pd.DataFrame.from_dict(gvkey_dict)
display(df_pred)

,61527,9715,1266,26753,28629,187406,10453,188157,183257,137432,9846,65671,10232,12216,20089,1718,112754,163118,180466,27965,144235,24832,2271,127282,19580,116504,20949,182701,14954,16582,8497,9372,111961,10247,146669,29143,64549,6565,63383,65142,175741,174663,10420,28317,176343,5977,27119,12304,3813,162198
0,0.372936,0.313453,0.361009,0.281325,0.335872,0.404536,0.358857,0.367288,0.302574,0.209094,0.198159,0.209285,0.215278,0.195743,0.352122,0.253826,0.348131,0.219558,0.269325,0.266666,0.347717,0.284155,0.271331,0.216798,0.271040,0.271145,0.195531,0.262811,0.308864,0.379743,0.263345,0.217624,0.251623,0.272728,0.321540,0.358869,0.277360,0.265544,0.277809,0.289896,0.320668,0.323520,0.282735,0.239620,0.207744,0.372803,0.330268,0.286602,0.276973,0.357234
1,0.247632,0.189511,0.294154,0.279754,0.259953,0.328689,0.277413,0.231324,0.182667,0.155974,0.161146,0.141840,0.202451,0.155033,0.245930,0.217812,0.306169,0.109960,0.157142,0.217545,0.298609,0.235861,0.213686,-0.018245,0.252988,0.157969,0.216606,0.195708,0.240630,0.348939,0.056084,0.097883,0.206605,0.235600,0.209253,0.224642,0.185036,0.172388,0.227713,0.277210,0.271938,0.247293,0.216092,0.222373,0.274162,0.357947,0.220470,0.170743,0.174281,0.243080
2,0.058523,0.162405,0.191868,0.113815,0.079654,0.211740,0.237915,0.120882,0.070739,-0.026516,0.046043,-0.003559,0.183392,0.007820,0.175308,0.073529,0.110757,0.064913,0.144177,0.068862,0.083351,0.177589,0.206479,-0.025643,0.194804,0.088915,0.103093,0.031859,0.095810,0.289610,-0.013152,0.126501,0.128803,0.042756,0.161213,0.113067,0.073122,0.063851,0.079764,0.139080,0.208870,0.161428,0.112787,0.222864,0.035919,0.247249,0.021399,0.147159,0.100773,0.243362
3,0.073664,0.114170,0.218529,0.122037,0.143496,0.203789,0.309487,0.211944,0.085880,0.067252,0.065528,-0.017248,0.367542,0.099089,0.249496,0.059909,0.128803,0.048902,0.216848,0.147681,0.112468,0.089752,0.166870,0.102114,0.103849,0.102405,0.152297,0.159705,0.167432,0.318652,0.080127,0.071383,0.054716,0.096657,0.195862,0.159630,0.051918,0.190569,0.156976,0.109007,0.352991,0.217857,0.109674,0.337363,0.136518,0.228692,0.079239,0.150963,0.194843,0.241494
4,0.015711,0.143272,0.137247,0.073499,0.076554,0.222898,0.284347,0.205669,0.013518,0.067326,0.024752,-0.021081,0.268664,0.075927,0.209668,-0.019635,0.114066,0.098024,0.184648,0.107594,0.083285,0.102307,0.142782,0.119033,0.125184,0.156690,0.207526,0.041764,0.139821,0.289175,0.106451,0.087394,0.075093,0.117162,0.169748,0.141135,-0.084092,0.101683,0.089265,0.112456,0.365963,0.126538,0.122090,0.276541,0.191310,0.177421,0.020299,0.030568,0.199866,0.235520
5,-0.079742,0.095236,-0.003763,-0.057279,-0.029884,0.083431,0.074889,0.124714,-0.068619,-0.067247,-0.100649,-0.155613,0.209775,-0.088601,0.112394,-0.070185,0.026698,-0.073622,0.109167,-0.038995,-0.073830,-0.020948,-0.025234,0.115233,-0.028842,0.092637,0.161262,-0.086275,-0.019982,0.164529,0.088248,0.023545,0.001334,0.011478,0.068365,0.021097,-0.109810,-0.032682,0.045321,0.140841,0.269546,0.064223,-0.035656,0.246404,0.032535,0.044332,-0.124832,0.029706,-0.002131,0.141588
6,0.069322,0.097980,0.084701,0.126759,0.068219,0.272452,0.165449,0.233261,0.105181,0.057028,0.031725,-0.016970,0.298982,0.036071,0.240418,-0.019781,0.095429,0.190542,0.131572,0.130716,0.047422,0.148510,0.115993,0.127789,0.106464,0.146827,0.224910,0.060305,0.047767,0.278523,0.236458,0.070263,0.122270,0.117528,0.154738,0.185960,-0.097929,0.077106,0.292083,0.353519,0.369430,0.189685,0.005874,0.318495,0.101868,0.226494,0.062536,0.286916,0.078775,0.296743
7,-0.152635,0.029626,-0.015289,0.061833,0.023036,0.263355,0.214671,0.009440,-0.003504,0.011736,0.056222,-0.075485,0.204089,-0.045828,0.235217,-0.048512,0.218550,0.180063,-0.109804,-0.045635,0.072894,0.137141,0.136315,0.086111,0.035894,0.150088,0.101508,0.067199,0.093556,0.156266,0.139910,-0.016109,-0.039529,0.019016,0.007419,0.188593,-0.058683,0.038524,0.062262,0.363722,0.312214,0.128012,-0.007997,0.350361,0.012420,0.187150,0.025412,0.268333,0.119426,0.245892
8,-0.395712,-0.079549,-0.2080

In [ ]:
# lm = LinearRegression()
# lm.fit(X_train, Y_train)
# pred_train = lm.predict(X_train)
# Y_pred = lm.predict(X_test)